# 전체 시스템 흐름 테스트

이 노트북은 DebateAgent를 사용한 전체 토론 시스템의 흐름을 테스트합니다.

## 테스트 흐름
1. **환경 설정**: 필요한 라이브러리 import 및 설정
2. **데이터 준비**: 데이터셋 빌드 (선택적)
3. **Round 0**: 초기 의견 수집 (각 에이전트의 독립적 예측)
4. **Round 1-N**: 반박(Rebuttal) 및 의견 수정(Revise)
5. **최종 결과**: Ensemble 예측 및 결과 분석


In [1]:
# 필요한 라이브러리 import
import sys
from pathlib import Path
from core.data_set import build_dataset

# 프로젝트 루트 경로 추가 (노트북 환경 대응)
notebook_dir = Path().resolve()  # 현재 노트북 디렉토리
project_root = notebook_dir.parent  # 프로젝트 루트 (notebooks의 상위 디렉토리)
sys.path.insert(0, str(project_root))

# 테스트 설정
TICKER = "NVDA"  # 테스트할 종목 티커
ROUNDS = 2  # 토론 라운드 수 (1-5 권장)

print(f"테스트 설정 완료")

테스트 설정 완료


In [2]:
from agents.macro_agent import MacroAgent
from agents.sentimental_agent import SentimentalAgent
from agents.technical_agent import TechnicalAgent
from agents.debate_agent import DebateAgent

In [3]:
debate = DebateAgent(rounds=ROUNDS, ticker=TICKER)

In [4]:
debate.run()

[18:03:22] 데이터셋 생성 중...
[TechnicalAgent] X_seq: (981, 55, 13), y_seq: (981, 1)
✅ NVDA TechnicalAgent dataset saved to CSV (981 samples, 13 features)
[MacroAgent] X_seq: (1195, 40, 13), y_seq: (1195, 1)
✅ NVDA MacroAgent dataset saved to CSV (1195 samples, 13 features)
[SentimentalAgent] X_seq: (1195, 40, 8), y_seq: (1195, 1)
✅ NVDA SentimentalAgent dataset saved to CSV (1195 samples, 8 features)
✅ NVDA TechnicalAgent dataset saved via technical_data_set


[*********************100%***********************]  15 of 15 completed


[MacroAgent] Data shape: (1302, 90), Columns: 90
[INFO] Feature engineering: 1302 rows, 90 features
[MacroAgent] Saved data/processed\NVDA_MacroAgent.csv


[*********************100%***********************]  1 of 1 completed


저장 완료: (1255, 1) rows
macro: 데이터셋 생성> NVDA
✅ NVDA MacroAgent dataset saved (macro_dataset 호출 via relative: .macro_classes.macro_funcs)
[SentimentalAgent] X_seq: (468, 14, 15), y_seq: (468, 1)
✅ NVDA None dataset saved to CSV (468 samples, 15 features)

[18:03:35] Round 0: 초기 Opinion 수집 시작
[18:03:35] [TechnicalAgent] pretrain 실행 (모델/스케일러 없음)
[18:03:36] Pretraining TechnicalAgent
  Epoch 005 | Loss: 0.398671
  Epoch 010 | Loss: 0.385365
  Epoch 015 | Loss: 0.387832
  Epoch 020 | Loss: 0.390167
  Epoch 025 | Loss: 0.385199
  Epoch 030 | Loss: 0.386073
  Epoch 035 | Loss: 0.378067
  Epoch 040 | Loss: 0.394221
  Epoch 045 | Loss: 0.407501
 TechnicalAgent 모델 학습 및 저장 완료: models\NVDA_TechnicalAgent.pt
[18:04:16] [TechnicalAgent] searcher 실행
[18:04:19] [TechnicalAgent] predict 실행
[18:04:21] [TechnicalAgent] reviewer_draft 실행
  - TechnicalAgent: next_close=186.6927
[18:04:43] [MacroAgent] pretrain 실행 (모델/스케일러 없음)
[18:04:43] Pretraining MacroAgent
[OK] MacroAData.model_maker: 데이터 + 시퀀스 + 스케일러 생성 

{'ticker': 'NVDA',
 'agents': {'TechnicalAgent_next_close': 186.5896104236528,
  'MacroAgent_next_close': 187.1011744361445,
  'SentimentalAgent_next_close': 185.7879726726591},
 'mean_next_close': 186.49291917748545,
 'median_next_close': 186.5896104236528,
 'currency': 'USD',
 'last_price': 186.52}

In [5]:
debate.opinions

{0: {'TechnicalAgent': Opinion(agent_id='TechnicalAgent', target=Target(next_close=186.69270479208643, uncertainty=0.005063805729150772, confidence=0.9949743568795758), reason='1) ma_200/2025-09-03 (0.1902×0.0502=0.0096): 200일 이동평균선은 장기적인 가격 흐름을 보여주며, 주가가 이 선 위에 있으면 상승 추세로 해석됩니다. 2) mom_10/2025-09-03 (0.1098×0.0502=0.0055): 10일 모멘텀은 최근 가격 변화 속도를 나타내어 단기 상승 모멘텀을 확인할 수 있습니다. 3) macd/2025-09-03 (0.0853×0.0502=0.0043): MACD는 추세의 강도와 방향을 파악하는 데 도움을 주며, 긍정적 신호는 상승 가능성을 시사합니다. 4) obv/2025-09-03 (0.0807×0.0502=0.0041): 거래량 기반 지표인 OBV는 매수세 강도를 반영하여 가격 상승을 뒷받침합니다. 상위 시점들이 2025-09-02~2025-09-03에 집중되어 있어 이 기간 동안 주가가 순매수 강화와 상방 추세를 보인 것으로 판단됩니다. 영향 점수 기준으로 ma_200이 가장 큰 역할을 하여 장기 상승 추세를 견고히 했고, 모멘텀과 MACD, OBV가 이를 보조하며 단기 상승 신호를 강화했습니다. 반면, bbp(0.0652×0.0502=0.0033)와 adx_14(0.0751×0.0502=0.0038) 등 일부 지표에서 과열 신호가 있었으나 영향 점수가 상대적으로 낮아 전체 상승 흐름에 큰 제약이 되지 않았습니다. 따라서 장기와 단기 지표가 조화를 이루며 2025-09-02~2025-09-03 기간 동안 안정적이고 긍정적인 상승 신호가 확인되어 다음날 종가 예측이 타당하다고 판단됩니다.'),
  'MacroAgent': Opinion(agent_id='MacroAgen

In [ ]:
# build_dataset(TICKER)

In [6]:
debate.get_opinion(0, TICKER)

[18:13:49] [TechnicalAgent] 기존 모델 사용: models\NVDA_TechnicalAgent.pt
[18:13:49] [TechnicalAgent] searcher 실행
[18:13:52] [TechnicalAgent] predict 실행
[18:13:52] [TechnicalAgent] reviewer_draft 실행
  - TechnicalAgent: next_close=186.6002
[18:14:09] [MacroAgent] 기존 모델 사용: models\NVDA_MacroAgent.pt
[18:14:09] [MacroAgent] searcher 실행
[INFO] 모델 및 스케일러 로드 중...
model_path: models\NVDA_MacroAgent.pt
[OK] 모델 및 스케일러 로드 완료
[INFO] MacroAgent 데이터 수집 중...
[INFO] Collecting macro features (15 tickers)...
[MacroAgent] Macro data: (44, 91)
[MacroAgent] Stock data: (43, 9)
[MacroAgent] Data shape: (43, 99)
[MacroAgent] Feature engineering complete. Final shape: (43, 170)
[OK] 매크로 데이터 수집 완료: (43, 171)
[INFO] 피처 정리 및 스케일링 중...
[Check] 입력 피처 수: 169 / 스케일러 기준 피처 수: 169
[OK] 스케일링 및 시퀀스 변환 완료
[18:14:12] [MacroAgent] predict 실행
[18:14:12] [MacroAgent] reviewer_draft 실행
  |  [INFO] Top 5 features: High_^IXIC=2.09e-04, Low_GC=F=1.63e-04, High_SPY=1.48e-04, Low_^GSPC=1.31e-04, Low_SPY=1.13e-04
  - MacroAgent: next_c

{'TechnicalAgent': Opinion(agent_id='TechnicalAgent', target=Target(next_close=186.60024808053154, uncertainty=0.004673889838159084, confidence=0.9953586415295124), reason='1) ma_200/2025-09-03 (0.1827×0.0502=0.0092): 200일 이동평균선은 장기적인 가격 추세를 보여주며, 이 지표가 강할수록 안정적인 상승세를 의미합니다. 2) mom_10/2025-09-03 (0.1141×0.0502=0.0057): 10일 모멘텀은 최근 가격 상승 속도를 나타내어 단기적인 상승 강도를 파악하는 데 유용합니다. 3) macd/2025-09-03 (0.0897×0.0502=0.0045): MACD는 추세의 전환점을 포착하는 데 도움을 주어 상승 신호를 확인할 수 있습니다. 4) adx_14/2025-09-03 (0.0763×0.0502=0.0038): ADX는 추세의 강도를 나타내며, 높은 값은 강한 추세 지속을 의미합니다. 상위 시점들이 모두 2025-09-03에 집중되어 있어, 2025-09-03~2025-09-05 기간 동안 강한 상승 추세가 형성되었음을 알 수 있습니다. 영향 점수 순으로 보면, ma_200이 0.0092로 가장 크고, 다음으로 mom_10(0.0057), macd(0.0045), adx_14(0.0038) 순으로 강한 상승 신호들이 일관되게 나타나 상승세를 뒷받침합니다. 반면, 과열이나 되돌림 신호는 상대적으로 낮은 영향 점수로 나타나 그 영향이 제한적임을 보여줍니다. 따라서 이 기간 동안 장기와 단기 지표 모두 상승 신호를 강하게 보내고 있어, 다음날 종가가 안정적으로 상승할 가능성이 높다고 판단됩니다.'),
 'MacroAgent': Opinion(agent_id='MacroAgent', target=Target(next_close=188.08958913855668, uncertain

In [7]:
debate.get_rebuttal(1)
debate.get_revise(1)

[18:14:30] [TechnicalAgent] → [MacroAgent] rebuttal 생성 중...
[18:14:38] [TechnicalAgent] → [SentimentalAgent] rebuttal 생성 중...
[18:14:46] [MacroAgent] → [TechnicalAgent] rebuttal 생성 중...
[18:14:55] [MacroAgent] → [SentimentalAgent] rebuttal 생성 중...
[18:15:05] [SentimentalAgent] → [TechnicalAgent] rebuttal 생성 중...
[warn] Non-numeric features dropped in load_dataset(): ['date']
[warn] Non-numeric features dropped in load_dataset(): ['date']
[FinBERT] 캐시 탐색: C:\Users\SPOTV\IdeaProjects\capstone_project\data\raw\news\NVDA.US_2025-10-09_2025-11-18.json (exists=True)
[FinBERT] 캐시 파일 로드 성공: 100건
[FinBERT] 100건 뉴스 감성 분석 시작...
[FinBERT] 7d_mean=-0.451 7d_cnt=100
[18:15:53] [SentimentalAgent] → [MacroAgent] rebuttal 생성 중...
[warn] Non-numeric features dropped in load_dataset(): ['date']
[warn] Non-numeric features dropped in load_dataset(): ['date']
[FinBERT] 캐시 탐색: C:\Users\SPOTV\IdeaProjects\capstone_project\data\raw\news\NVDA.US_2025-10-09_2025-11-18.json (exists=True)
[FinBERT] 캐시 파일 로드 성공: 1

{'TechnicalAgent': Opinion(agent_id='TechnicalAgent', target=Target(next_close=186.60874440370964, uncertainty=0.004185783211141825, confidence=0.9958403279562505), reason='기존 의견에서는 200일 이동평균선과 10일 모멘텀, MACD, ADX 등 주요 기술 지표들이 모두 강한 상승 신호를 보내고 있어, 다음날 종가가 현재가 대비 상승할 가능성이 높다는 점을 중심으로 예측하셨습니다. 즉, 장기와 단기 모두 상승 추세에 무게를 두고 계셨습니다. 동료 에이전트들의 의견을 보면 MacroAgent는 기술적 지표와 거시경제 환경이 긍정적이며, 이동평균선과 거래량 증가, RSI 지표가 추가 상승 여력을 시사해 공격적이면서도 높은 수준의 상승 예측을 제시했습니다. 반면 SentimentalAgent는 최근 투자자 심리가 부정적이고 긍정 뉴스 비율이 낮아 단기 상승에 불확실성이 있다고 반박하였습니다. 이 중 MacroAgent의 거시경제 및 기술적 지표 연계 분석은 신뢰도가 높아 수용하였으나, SentimentalAgent의 부정적 감성 신호는 기술적 모멘텀과 추세 신호에 비해 상대적으로 약한 영향으로 판단하여 보류하였습니다. 기술적 신호를 요약하면, 가격·추세 측면에서 200일 이동평균선이 꾸준히 상승 중이며 MACD도 상승세를 유지해 단기와 중기 모두 상승 추세를 따르고 있습니다. 모멘텀 지표인 10일 모멘텀은 최근 다소 변동성이 있으나 전반적으로 상승세를 지지하며, 거래량은 20일 이동평균 대비 안정적이고 변동성도 큰 폭으로 확대되지 않아 과열 신호는 미미합니다. 단기와 중기 흐름이 일치하는 점이 긍정적입니다. 따라서 이번 수정에서는 기존 예측보다 다소 공격적으로 상승 쪽에 무게를 더 두는 방향으로 조정하였습니다. 이는 MacroAgent의 거시경제 호조 신호와 기술적 지표의 일관된 상승 신호를 반영한 결과입니다. 다만 Sentime

In [8]:
debate.get_rebuttal(2)
debate.get_revise(2)

[18:18:31] [TechnicalAgent] → [MacroAgent] rebuttal 생성 중...
[18:18:40] [TechnicalAgent] → [SentimentalAgent] rebuttal 생성 중...
[18:18:49] [MacroAgent] → [TechnicalAgent] rebuttal 생성 중...
[18:19:07] [MacroAgent] → [SentimentalAgent] rebuttal 생성 중...
[18:19:19] [SentimentalAgent] → [TechnicalAgent] rebuttal 생성 중...
[warn] Non-numeric features dropped in load_dataset(): ['date']
[warn] Non-numeric features dropped in load_dataset(): ['date']
[FinBERT] 캐시 탐색: C:\Users\SPOTV\IdeaProjects\capstone_project\data\raw\news\NVDA.US_2025-10-09_2025-11-18.json (exists=True)
[FinBERT] 캐시 파일 로드 성공: 100건
[FinBERT] 100건 뉴스 감성 분석 시작...
[FinBERT] 7d_mean=-0.451 7d_cnt=100
[18:20:18] [SentimentalAgent] → [MacroAgent] rebuttal 생성 중...
[warn] Non-numeric features dropped in load_dataset(): ['date']
[warn] Non-numeric features dropped in load_dataset(): ['date']
[FinBERT] 캐시 탐색: C:\Users\SPOTV\IdeaProjects\capstone_project\data\raw\news\NVDA.US_2025-10-09_2025-11-18.json (exists=True)
[FinBERT] 캐시 파일 로드 성공: 1

{'TechnicalAgent': Opinion(agent_id='TechnicalAgent', target=Target(next_close=186.6008331645839, uncertainty=0.004674247931689024, confidence=0.9953582884030127), reason='기존 의견에서는 200일 이동평균선과 10일 모멘텀, MACD, ADX 등 주요 기술 지표들이 모두 강한 상승 신호를 보내고 있어 다음날 종가가 현재가 대비 상승할 가능성이 높다는 점에 무게를 두셨고, 장기와 단기 모두 상승 추세에 무게를 두셨습니다. 동료 에이전트들의 의견을 보면 MacroAgent는 금리 인상 기조 둔화와 인플레이션 압력 완화, 유동성 회복세 등 거시경제 환경이 긍정적이며, 기술적 지표와 거래량 증가, RSI 지표가 추가 상승 여력을 시사해 공격적이고 높은 수준의 상승 예측을 제시한 반면, SentimentalAgent는 최근 투자자 심리가 부정적이고 부정 기사 비율이 높아 단기 상승에 불확실성이 있다고 반박하였습니다. 이 중 MacroAgent의 거시경제 및 기술적 지표 연계 분석은 신뢰도가 높아 수용하였으나, SentimentalAgent의 부정적 감성 신호는 기술적 모멘텀과 추세 신호에 비해 영향력이 제한적이라 판단하여 보류하였습니다. 기술적 신호를 요약하면, 가격·추세 측면에서 200일 이동평균선이 꾸준히 상승 중이며 MACD도 상승세를 유지해 단기와 중기 모두 상승 추세를 따르고 있습니다. 모멘텀 지표인 10일 모멘텀은 최근 변동성이 있으나 전반적으로 상승세를 지지하며, 거래량은 20일 이동평균 대비 안정적이고 변동성도 크게 확대되지 않아 과열 신호는 미미합니다. 단기와 중기 흐름이 일치하는 점이 긍정적입니다. 따라서 이번 수정에서는 기존 예측보다 다소 공격적으로 상승 쪽에 무게를 더 두는 방향으로 조정하였으며, 이는 MacroAgent의 거시경제 호조 신호와 기술적 지표의 일관된 상승 신호를 반영한 결과입니다. 다만 Sentim

In [ ]:
macro = MacroAgent(ticker=TICKER)
technical = TechnicalAgent(ticker=TICKER)
sentimental = SentimentalAgent(ticker=TICKER)

In [ ]:
data = technical.searcher()

In [ ]:
technical.pretrain()

In [ ]:
technical.predict(data)

In [ ]:
technical.reviewer_draft()

## 1. DebateAgent 초기화

세 가지 에이전트를 포함한 DebateAgent를 초기화합니다:
- **TechnicalAgent**: 기술적 분석가
- **MacroSentiAgent**: 거시경제 분석가  
- **SentimentalAgent**: 센티멘탈 분석가


In [ ]:
# DebateAgent 초기화
print("=" * 60)
print("■ DebateAgent 초기화 중...")
print("=" * 60)

debate_agent = DebateAgent(rounds=ROUNDS, ticker=TICKER)

print(f"\n✅ DebateAgent 초기화 완료")
print(f"   - 에이전트 수: {len(debate_agent.agents)}")
print(f"   - 에이전트 목록: {list(debate_agent.agents.keys())}")
print(f"   - 토론 라운드: {debate_agent.rounds}")


## 2. 데이터셋 준비 (선택적)

데이터셋이 이미 존재하는 경우 이 단계를 건너뛸 수 있습니다.


In [ ]:
# 데이터셋 빌드 (필요한 경우에만 실행)
# 주석을 해제하여 실행하세요
print("=" * 60)
print("■ 데이터셋 빌드 중...")
print("=" * 60)
build_dataset(TICKER)
print("✅ 데이터셋 빌드 완료\n")


## 3. Round 0: 초기 의견 수집

각 에이전트가 독립적으로 주식을 분석하고 예측합니다.


In [ ]:
# Round 0: 초기 의견 수집
print("=" * 60)
print("■ Round 0: 초기 의견 수집 시작")
print("=" * 60)

opinions_0 = debate_agent.get_opinion(round=0, ticker=TICKER, rebuild=True, force_pretrain=False)

print("\n" + "=" * 60)
print("✅ Round 0 완료: 초기 의견 수집 결과")
print("=" * 60)

for agent_id, opinion in opinions_0.items():
    if opinion and opinion.target:
        print(f"\n[{agent_id}]")
        print(f"  예측 종가: ${opinion.target.next_close:.2f}")
        print(f"  불확실성(σ): {opinion.target.uncertainty:.6f}" if opinion.target.uncertainty else "  불확실성: N/A")
        print(f"  신뢰도(β): {opinion.target.confidence:.4f}" if opinion.target.confidence else "  신뢰도: N/A")
        print(f"  근거 요약: {opinion.reason[:100]}..." if len(opinion.reason) > 100 else f"  근거: {opinion.reason}")


##[10:40:57] [MacroAgent] predict 실행
[WARN] 모델 로드 실패: Error(s) in loading state_dict for MacroAgent:
	size mismatch for lstm1.weight_ih_l0: copying a param with shape torch.Size([512, 2]) from checkpoint, the shape in current model is torch.Size([512, 13])., pretrain 수행...

## 4. Round 1-N: 반박 및 의견 수정

각 라운드마다:
1. **Rebuttal**: 다른 에이전트의 의견에 대한 반박/지지
2. **Revise**: 신뢰도 기반으로 자신의 의견 수정


In [ ]:
# Round 1-N: 반박 및 의견 수정
for round_num in range(1, ROUNDS + 1):
    print("\n" + "=" * 60)
    print(f"■ Round {round_num}: 반박 및 의견 수정")
    print("=" * 60)
    
    # 4-1. Rebuttal (반박)
    print(f"\n[Round {round_num}-1] Rebuttal 수행 중...")
    rebuttals = debate_agent.get_rebuttal(round=round_num)
    print(f"✅ Rebuttal 완료: {len(rebuttals)}개의 반박 생성")
    
    # 반박 내용 일부 출력
    for rebuttal in rebuttals[:2]:  # 처음 2개만 출력
        print(f"\n  [{rebuttal.from_agent_id} → {rebuttal.to_agent_id}]")
        print(f"    스탠스: {rebuttal.stance}")
        print(f"    메시지: {rebuttal.message[:150]}...")
    
    # 4-2. Revise (의견 수정)
    print(f"\n[Round {round_num}-2] Revise 수행 중...")
    revised_opinions = debate_agent.get_revise(round=round_num)
    print(f"✅ Revise 완료: {len(revised_opinions)}개 에이전트 의견 수정")
    
    # 수정된 의견 출력
    print(f"\n[Round {round_num}] 수정된 의견:")
    for agent_id, opinion in revised_opinions.items():
        if opinion and opinion.target:
            prev_opinion = debate_agent.opinions[round_num - 1][agent_id]
            prev_price = prev_opinion.target.next_close if prev_opinion and prev_opinion.target else None
            
            change = opinion.target.next_close - prev_price if prev_price else 0
            change_pct = (change / prev_price * 100) if prev_price else 0
            
            print(f"\n  [{agent_id}]")
            print(f"    이전 예측: ${prev_price:.2f}" if prev_price else "    이전 예측: N/A")
            print(f"    수정 예측: ${opinion.target.next_close:.2f}")
            print(f"    변화: ${change:+.2f} ({change_pct:+.2f}%)")
            print(f"    불확실성: {opinion.target.uncertainty:.6f}" if opinion.target.uncertainty else "    불확실성: N/A")
    
    print(f"\n✅ Round {round_num} 토론 완료")


[MacroAgent] revision 메시지 생성 실패: 'NoneType' object has no attribute 'upper'


In [ ]:
# Round 1-N: 반박 및 의견 수정
for round_num in range(1, ROUNDS + 1):
    print("\n" + "=" * 60)
    print(f"■ Round {round_num}: 반박 및 의견 수정")
    print("=" * 60)
    
    # 4-1. Rebuttal (반박)
    print(f"\n[Round {round_num}-1] Rebuttal 수행 중...")
    rebuttals = debate_agent.get_rebuttal(round=round_num)
    print(f"✅ Rebuttal 완료: {len(rebuttals)}개의 반박 생성")
    
    # 반박 내용 일부 출력
    for rebuttal in rebuttals[:2]:  # 처음 2개만 출력
        print(f"\n  [{rebuttal.from_agent_id} → {rebuttal.to_agent_id}]")
        print(f"    스탠스: {rebuttal.stance}")
        print(f"    메시지: {rebuttal.message[:150]}...")
    
    # 4-2. Revise (의견 수정)
    print(f"\n[Round {round_num}-2] Revise 수행 중...")
    revised_opinions = debate_agent.get_revise(round=round_num)
    print(f"✅ Revise 완료: {len(revised_opinions)}개 에이전트 의견 수정")
    
    # 수정된 의견 출력
    print(f"\n[Round {round_num}] 수정된 의견:")
    for agent_id, opinion in revised_opinions.items():
        if opinion and opinion.target:
            prev_opinion = debate_agent.opinions[round_num - 1][agent_id]
            prev_price = prev_opinion.target.next_close if prev_opinion and prev_opinion.target else None
            
            change = opinion.target.next_close - prev_price if prev_price else 0
            change_pct = (change / prev_price * 100) if prev_price else 0
            
            print(f"\n  [{agent_id}]")
            print(f"    이전 예측: ${prev_price:.2f}" if prev_price else "    이전 예측: N/A")
            print(f"    수정 예측: ${opinion.target.next_close:.2f}")
            print(f"    변화: ${change:+.2f} ({change_pct:+.2f}%)")
            print(f"    불확실성: {opinion.target.uncertainty:.6f}" if opinion.target.uncertainty else "    불확실성: N/A")
    
    print(f"\n✅ Round {round_num} 토론 완료")


## 5. 최종 결과: Ensemble 예측

모든 라운드가 완료된 후 최종 예측 결과를 확인합니다.


In [ ]:
# 최종 결과: Ensemble 예측
print("\n" + "=" * 60)
print("■ 최종 결과: Ensemble 예측")
print("=" * 60)

ensemble_result = debate_agent.get_ensemble()

print("\n📊 최종 예측 결과:")
print(f"  종목: {ensemble_result['ticker']}")
print(f"  통화: {ensemble_result['currency']}")
print(f"  현재가: ${ensemble_result['last_price']:.2f}" if ensemble_result['last_price'] else "  현재가: N/A")
print(f"\n  에이전트별 예측:")
for key, value in ensemble_result['agents'].items():
    agent_name = key.replace('_next_close', '')
    print(f"    {agent_name}: ${value:.2f}")

print(f"\n  앙상블 예측:")
print(f"    평균: ${ensemble_result['mean_next_close']:.2f}" if ensemble_result['mean_next_close'] else "    평균: N/A")
print(f"    중앙값: ${ensemble_result['median_next_close']:.2f}" if ensemble_result['median_next_close'] else "    중앙값: N/A")

# 현재가와 비교
if ensemble_result['last_price'] and ensemble_result['mean_next_close']:
    current = ensemble_result['last_price']
    predicted = ensemble_result['mean_next_close']
    change = predicted - current
    change_pct = (change / current) * 100
    print(f"\n  예상 수익률:")
    print(f"    절대 변화: ${change:+.2f}")
    print(f"    상대 변화: {change_pct:+.2f}%")


## 6. 라운드별 의견 변화 시각화

각 라운드별로 에이전트의 예측이 어떻게 변화했는지 확인합니다.


In [ ]:
# 라운드별 의견 변화 분석
import pandas as pd
import matplotlib.pyplot as plt

print("\n" + "=" * 60)
print("■ 라운드별 의견 변화 분석")
print("=" * 60)

# 라운드별 의견 데이터 수집
rounds_data = []
for round_num in sorted(debate_agent.opinions.keys()):
    opinions = debate_agent.opinions[round_num]
    for agent_id, opinion in opinions.items():
        if opinion and opinion.target:
            rounds_data.append({
                'Round': round_num,
                'Agent': agent_id,
                'Predicted_Price': opinion.target.next_close,
                'Uncertainty': opinion.target.uncertainty or 0,
                'Confidence': opinion.target.confidence or 0
            })

df_rounds = pd.DataFrame(rounds_data)

if not df_rounds.empty:
    # 라운드별 예측가 변화 시각화
    plt.figure(figsize=(12, 6))
    
    for agent_id in df_rounds['Agent'].unique():
        agent_data = df_rounds[df_rounds['Agent'] == agent_id].sort_values('Round')
        plt.plot(agent_data['Round'], agent_data['Predicted_Price'], 
                marker='o', label=agent_id, linewidth=2, markersize=8)
    
    plt.xlabel('Round', fontsize=12)
    plt.ylabel('Predicted Price ($)', fontsize=12)
    plt.title(f'Round별 예측가 변화 ({TICKER})', fontsize=14, fontweight='bold')
    plt.legend(loc='best', fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # 테이블로도 출력
    print("\n📋 라운드별 예측가 변화 테이블:")
    pivot_table = df_rounds.pivot_table(
        index='Agent', 
        columns='Round', 
        values='Predicted_Price',
        aggfunc='first'
    )
    print(pivot_table.round(2))
else:
    print("데이터가 없습니다.")


## 7. 간편 실행: run() 메서드 사용

위의 단계를 한 번에 실행하려면 `run()` 메서드를 사용할 수 있습니다.


In [ ]:
# 간편 실행 (주석 해제하여 사용)
# debate_agent_simple = DebateAgent(rounds=ROUNDS, ticker=TICKER)
# debate_agent_simple.run()
# print(debate_agent_simple.get_ensemble())


## 8. 전체 디베이트 실행 및 문제점 진단

`run()` 메서드를 사용하여 전체 디베이트를 한 번에 실행하고 문제점을 진단합니다.


In [ ]:
# 전체 디베이트 실행 및 문제점 진단
import traceback
from datetime import datetime

print("=" * 80)
print("전체 디베이트 실행 시작")
print("=" * 80)
print(f"시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

# 새로운 DebateAgent 인스턴스 생성
debate_full = DebateAgent(rounds=ROUNDS, ticker=TICKER)

try:
    print("1️⃣ run() 메서드 실행 중...")
    print("-" * 80)
    debate_full.run()
    print("\n✅ run() 메서드 완료!")
    
    print("\n2️⃣ 최종 결과 확인...")
    print("-" * 80)
    ensemble = debate_full.get_ensemble()
    print("\n📊 최종 Ensemble 결과:")
    for key, value in ensemble.items():
        if isinstance(value, dict):
            print(f"  {key}:")
            for k, v in value.items():
                print(f"    {k}: {v}")
        else:
            print(f"  {key}: {value}")
            
except Exception as e:
    print(f"\n❌ 오류 발생: {type(e).__name__}: {e}")
    print("\n" + "=" * 80)
    print("스택 트레이스:")
    print("=" * 80)
    traceback.print_exc()
    
    # 현재 상태 확인
    print("\n" + "=" * 80)
    print("현재 상태 확인:")
    print("=" * 80)
    print(f"  opinions keys: {list(debate_full.opinions.keys())}")
    print(f"  rebuttals keys: {list(debate_full.rebuttals.keys())}")
    if debate_full.opinions:
        print(f"  마지막 라운드: {max(debate_full.opinions.keys())}")
        last_round = max(debate_full.opinions.keys())
        print(f"  마지막 라운드 의견 수: {len(debate_full.opinions[last_round])}")


## 완료

전체 토론 시스템 테스트가 완료되었습니다! 🎉

### 주요 결과 요약:
- ✅ 3개 에이전트의 초기 의견 수집 완료
- ✅ {ROUNDS}라운드의 반박 및 의견 수정 완료
- ✅ 최종 앙상블 예측 생성 완료

### 다음 단계:
- 다른 종목으로 테스트해보기
- 라운드 수를 조정하여 수렴 패턴 관찰
- Streamlit 대시보드에서 인터랙티브하게 테스트
